<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Simple Linear Regression with Sacramento Real Estate Data

_Authors: Matt Brems, Sam Stack_

---

In this lab you will hone your exploratory data analysis (EDA) skills and practice constructing simple linear regressions using a data set on Sacramento real estate sales.  The data set contains information on qualities of the property, location of the property, and time of sale.

### 1. Read in the Sacramento housing data set.

In [1]:
sac_csv = '../datasets/sacramento_real_estate_transactions.csv'

In [2]:
import pandas as pd

shd = pd.read_csv(sac_csv)

shd.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


### 2. Conduct exploratory data analysis on this data set. 

**Report any notable findings here and any steps you take to clean/process data.**

> **Note:** These EDA checks should be done on every data set you handle. If you find yourself checking repeatedly for missing/corrupted data, it might be beneficial to have a function that you can reuse every time you're given new data.

In [3]:
# Check the dtypes.
shd.dtypes

street        object
city          object
zip            int64
state         object
beds           int64
baths          int64
sq__ft         int64
type          object
sale_date     object
price          int64
latitude     float64
longitude    float64
dtype: object

In [4]:
# Zip code works better as a string object (categorical) so we will have to convert it.
shd['zip'] = shd['zip'].astype(str)

_**Fun Fact:** Zip codes often have leading zeros — e.g., 02215 = Boston, MA — which will often get knocked off automatically by many software programs like Python or Excel. You can imagine that this could create some issues. _

In [5]:
# Check out the summary statistics:
shd.describe()

,beds,baths,sq__ft,price,latitude,longitude
count,985.000000,985.000000,985.000000,985.000000,985.000000,985.000000
mean,2.911675,1.776650,1312.918782,233715.951269,38.445121,-121.193371
std,1.307932,0.895371,856.123224,139088.818896,5.103637,5.100670
min,0.000000,0.000000,-984.000000,-210944.000000,-121.503471,-121.551704
25%,2.000000,1.000000,950.000000,145000.000000,38.482704,-121.446119
50%,3.000000,2.000000,1304.000000,213750.000000,38.625932,-121.375799
75%,4.000000,2.000000,1718.000000,300000.000000,38.695589,-121.294893
max,8.000000,5.000000,5822.000000,884790.000000,39.020808,38.668433


In [6]:
# It appears that we have some values that are out of place, as there are
# houses with 0 bedrooms, 0 baths, negative square footage, and negative prices.  
# There are also some bizarre longitudes/latitudes in our data set. A house in Antarctica perhaps? 

In [7]:
# Check out the cities. Most cities have very few observations.
shd.city.value_counts()

SACRAMENTO         439
ELK GROVE          114
LINCOLN             72
ROSEVILLE           48
CITRUS HEIGHTS      35
ANTELOPE            33
RANCHO CORDOVA      28
EL DORADO HILLS     23
GALT                21
NORTH HIGHLANDS     21
CARMICHAEL          20
ROCKLIN             17
FOLSOM              17
RIO LINDA           13
ORANGEVALE          11
PLACERVILLE         10
CAMERON PARK         9
FAIR OAKS            9
WILTON               5
AUBURN               5
ELVERTA              4
GOLD RIVER           4
POLLOCK PINES        3
GRANITE BAY          3
RANCHO MURIETA       3
WEST SACRAMENTO      3
LOOMIS               2
EL DORADO            2
MATHER               1
MEADOW VISTA         1
DIAMOND SPRINGS      1
PENRYN               1
COOL                 1
WALNUT GROVE         1
GREENWOOD            1
SLOUGHHOUSE          1
SHINGLE SPRINGS      1
GARDEN VALLEY        1
FORESTHILL           1
Name: city, dtype: int64

In [8]:
# What could explain the houses with 0 bedrooms?
shd[shd['beds'] == 0]

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
73,17 SERASPI CT,SACRAMENTO,95834,CA,0,0,0,Residential,Wed May 21 00:00:00 EDT 2008,206000,38.631481,-121.501880
89,2866 KARITSA AVE,SACRAMENTO,95833,CA,0,0,0,Residential,Wed May 21 00:00:00 EDT 2008,244500,38.626671,-121.525970
100,12209 CONSERVANCY WAY,RANCHO CORDOVA,95742,CA,0,0,0,Residential,Wed May 21 00:00:00 EDT 2008,263500,38.553867,-121.219141
121,5337 DUSTY ROSE WAY,RANCHO CORDOVA,95742,CA,0,0,0,Residential,Wed May 21 00:00:00 EDT 2008,320000,38.528575,-121.228600
126,2115 SMOKESTACK WAY,SACRAMENTO,95833,CA,0,0,0,Residential,Wed May 21 00:00:00 EDT 2008,339500,38.602416,-121.542965
...,...,...,...,...,...,...,...,...,...,...,...,...
901,1530 TOPANGA LN Unit 204,LINCOLN,95648,CA,0,0,0,Condo,Thu May 15 00:00:00 EDT 2008,138000,38.884150,-121.270277
917,501 POPLAR AVE,WEST SACRAMENTO,95691,CA,0,0,0,Residential,Thu May 15 00:00:00 EDT 2008,165000,38.584526,-121.534609
934,1550 TOPANGA LN Unit 207,LINCOLN,95648,CA,0,0,0,Condo,Thu May 15 00:00:00 EDT 2008,188000,38.884170,-121.270222
947,1525 PENNSYLVANIA AVE,WEST SACRAMENTO,95691,CA,0,0,0,Residential,Thu May 15 00:00:00 EDT 2008,200100,38.569943,-121.527539


In [9]:
print(shd[shd['beds'] == 0].shape)

(108, 12)


In [10]:
# Given the large number of houses that have 0 beds, 0 baths and 0 square feet, 
# we can make an assumption that these are plots of land that haven't had anything built on them yet.
# As a result we will *not* be dropping them.

In [11]:
# What about the houses that are less than 0 dollars?
shd[shd['price'] < 1]

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
703,1900 DANBROOK DR,SACRAMENTO,95835,AC,1,1,-984,Condo,Fri May 16 00:00:00 EDT 2008,-210944,-121.503471,38.668433


In [12]:
# And the houses with negative square footage?
shd[shd['sq__ft'] < 0]

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
703,1900 DANBROOK DR,SACRAMENTO,95835,AC,1,1,-984,Condo,Fri May 16 00:00:00 EDT 2008,-210944,-121.503471,38.668433


In [13]:
# It looks like the house with a negative price is also the one with negative square footage.
# It is time to make a choice: Either assume that the data were entered improperly and are meant 
# to be positive, or drop the data altogether.

# Side note: Notice the state is actually labeled incorrectly as well.

In [14]:
# Let's check if any other values are labeled incorrectly.
shd[shd['state'] != 'CA']

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
703,1900 DANBROOK DR,SACRAMENTO,95835,AC,1,1,-984,Condo,Fri May 16 00:00:00 EDT 2008,-210944,-121.503471,38.668433


In [15]:
# It looks like only one row is mislabeled, so we'll just drop that row.

shd.drop(703, inplace = True)

### 3. Our goal will be to predict price. List variables that you think qualify as predictors of price in an SLR model. 

**For each of the variables you believe to be a valid potential predictor in an SLR model, generate a plot showing the relationship between the independent and dependent variables.**

In [16]:
# 'Beds','baths', and'sq__ft' seem like they would be valid potential predictors.
# 'Latitude' and 'longitude' are other options to consider, but those are more involved.

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

sns.lmplot(x='sq__ft', y='price', data=shd)
plt.show()
sns.lmplot(x='beds', y='price', data=shd)
plt.show()
sns.lmplot(x='baths', y='price', data=shd)
plt.show()

UsageError: Line magic function `%` not found.


When you've finished cleaning or have made a good deal of progress cleaning, it's always a good idea to save your work.
```python
shd.to_csv('./datasets/sacramento_real_estate_transactions_Clean.csv')
```

### 4. Which variable would be the best predictor of Y in an SLR model? Why?

In [ ]:
# It looks like square footage is a better predictor of price than number of beds or baths.
# Beds and baths are discrete measures as opposed to square footage, which is a continuous measure.  
# Additionally, there are likely strong correlations between the square footage and numbers of bedrooms and bathrooms, in that houses 
# with greater square footage will likely have more bedrooms and bathrooms.

# If we dropped all of the plots of land that are in the data set (those with 0 sq ft, 
# beds, and baths), we would see a much stronger trend line in our lm plot.

### 5. Build a function that will take in two lists, `Y` and `X`, and return the intercept and slope coefficients that minimize SSE. 

`Y` is the target variable and `X` is the predictor variable.

- **Test your function on price and the variable you determined was the best predictor in Problem 4.**
- **Report the slope and intercept.**

In [ ]:
import numpy as np
import scipy.stats

# Get the optimal slope and y intercept.

def lin_reg(x,y):
    # Using other libraries for standard deviation and the Pearson correlation coefficient.
    # Note that in SLR, the correlation coefficient multiplied by the standard
    # deviation of y divided by standard deviation of x is the optimal slope.
    beta_1 = (scipy.stats.pearsonr(x,y)[0])*(np.std(y)/np.std(x))
    
    # The Pearson correlation coefficient returns a tuple, so it needs to be sliced/indexed.
    # The optimal beta is found by: mean(y) - b1 * mean(x).
    beta_0 = np.mean(y)-(beta_1*np.mean(x)) 
    
    return beta_0, beta_1

In [ ]:
x = shd['sq__ft'].values
y = shd['price'].values
beta0, beta1 = lin_reg(x,y)

#Print the optimal values.
print('The Optimal Y Intercept is ', beta0)
print('The Optimal slope is ', beta1)

### 6. Interpret the intercept. Interpret the slope.

In [ ]:
# The intercept should be the relative y value around which our data begin 
# in that from this point out, as the x values increase, so should the y value from this point.

# The intercept is an "offset." Without the intercept, our regression line would
# be forced to pass through the origin.

# The slope is the increase in our target variable (price) for every 1-unit increase
# in our predictor variable (sq__ft). So in our example, for every sq__ft increase there is
# an associated increase of ~54 dollars in price. 

### 7. Give an example of how this model could be used for prediction and how it could be used for inference. 

**Be sure to make it clear which example is associated with prediction and which is associated with inference.**

In [ ]:
# Prediction:
# You are a real estate agent with a separate database on house characteristics and locations.
# You want to estimate what price one of these houses will most likely sell for based
# on the model that we built using this data.

# Inference:
# You work for an architecture company that wants to understand what specific locations and housing characteristics 
# are associated with perceived value. You have some hypotheses about what
# makes a house valuable but you would like to test these hypotheses.

### 8: [Bonus] Using the model you came up with in Problem 5, calculate and plot the residuals.

In [ ]:
# Predictor: y = 162938.74 + 54.16x
# Creating a list of predicted values:

y_pred = beta0 + beta1*x

In [ ]:
# Appending the predicted values to the Sacramento housing DataFrame to do DF calcs:
shd['Pred'] = y_pred

# Residuals equals the difference between Y-True and Y-Pred:
shd['Residuals'] = abs(shd['price']-shd['Pred'])

In [ ]:
shd['Residuals'].mean()
# The mean of our residuals is approximately $96,000, which means that 
# on average our prediction is off by this amount.

In [ ]:
# Plot showing our linear forecast.
fig = plt.figure(figsize=(20,20))

# Change the font size of minor ticks label.
plot = fig.add_subplot(111)
plot.tick_params(axis='both', which='major', labelsize=20)

# Get the axis of that figure.
ax = plt.gca()

# Plot a scatterplot on the axis using our data.
ax.scatter(x= shd['sq__ft'], y=shd['price'], c='k')
ax.plot(shd['sq__ft'], shd['Pred'], color='r');

> Bonus: It looks like all of those "zero-price" entries are skewing our predictions.  We could correct for that!  How

In [ ]:
# Plot with residuals.
fig = plt.figure(figsize=(20,20))

# Change the font size of minor ticks label.
plot = fig.add_subplot(111)
plot.tick_params(axis='both', which='major', labelsize=20)

# Get the axis of that figure.
ax = plt.gca()

# Plot a scatterplot on the axis using our data.
ax.scatter(x= shd['sq__ft'], y=shd['price'], c='k')
ax.plot(shd['sq__ft'], shd['Pred'], color='r');

# Iterate over predictions.
for _, row in shd.iterrows():
    plt.plot((row['sq__ft'], row['sq__ft']), (row['price'], row['Pred']), 'b-')

In [ ]:
# One more plot; let's look at how our predictions compare to the true values.
sns.lmplot(x='price', y='Pred', data=shd)

In [ ]:
# Given our last visual, we can see that a lot of points were plotted along the 
# y intercept (y= 162938.74). These were all the houses reported to have a
# square footage of 0. We could probably create a more insightful
# model if we removed those observations from our data.

---

> The material following this point can be completed after the second lesson on Monday.

---

## Dummy Variables

---

It is important to be cautious with categorical variables, which represent distict groups or categories, when building a regression. If put in a regression "as-is," categorical variables represented as integers will be treated like *continuous* variables.

That is to say, instead of group "3" having a different effect on the estimation than group "1" it will estimate literally 3 times more than group 1. 

For example, if occupation category "1" represents "analyst" and occupation category "3" represents "barista", and our target variable is salary, if we leave this as a column of integers then barista will always have `beta*3` the effect of analyst.

This will almost certainly force the beta coefficient to be something strange and incorrect. Instead, we can re-represent the categories as multiple "dummy coded" columns.

### 9. Use the `pd.get_dummies` function to convert the `type` column into dummy-coded variables.

Print out the header of the dummy-coded variable output.

In [ ]:
shd['type'].unique()

In [ ]:
type_dummy = pd.get_dummies(shd['type'])
type_dummy.head()

---

### A Word of Caution When Creating Dummies

Let's touch on precautions we should take when dummy coding.

**If you convert a qualitative variable to dummy variables, you want to turn a variable with N categories into N-1 variables.**

> **Scenario 1:** Suppose we're working with the variable "sex" or "gender" with values "M" and "F". 

You should include in your model only one variable for "sex = F" which takes on 1 if sex is female and 0 if sex is not female! Rather than saying "a one unit change in X," the coefficient associated with "sex = F" is interpreted as the average change in Y when sex = F relative to when sex = M.

| Female | Male | 
|-------|------|
| 0 | 1 | 
| 1 | 0 |
| 0 | 1 |
| 1 | 0 |
| 1 | 0 |
_As we can see a 1 in the female column indicates a 0 in the male column. And so, we have two columns stating the same information in different ways._

> Scenario 2: Suppose we're modeling revenue at a bar for each of the days of the week. We have a column with strings identifying which day of the week this observation occured in.

We might include six of the days as their own variables: "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday". **But not all 7 days.**  

|Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | 
|-------|---------|-----------|----------|--------|----------|
| 1     | 0       |0          |      0   |0       | 0        | 
| 0     | 1       |0          |      0   |0       | 0        | 
| 0     | 0       |1          |      0   |0       | 0        | 
| 0     | 0       |0          |      1   |0       | 0        | 
| 0     | 0       |0          |      0   |1       | 0        | 
| 0     | 0       |0          |      0   |0       | 1        | 
| 0     | 0       |0          |      0   |0       | 0        | 

_As humans we can infer from the last row that if its is not Monday, Tusday, Wednesday, Thursday, Friday or Saturday than it must be Sunday. Models work the same way._

The coefficient for Monday is then interpreted as the average change in revenue when "day = Monday" relative to "day = Sunday." The coefficient for Tuesday is interpreted in the average change in revenue when "day = Tuesday" relative to "day = Sunday" and so on.

The category you leave out, which the other columns are *relative to* is often referred to as the **reference category**.

### 10. Remove "Unkown" from four dummy coded variable dataframe and append the rest to the original data.

In [ ]:
type_dummy.drop('Unkown', axis=1, inplace=True)
shd = pd.concat([shd, type_dummy], axis=1)
shd.head(2)

### 11. Build what you think may be the best MLR model predicting `price`. 

The independent variables are your choice, but *include at least three variables.* At least one of which should be a dummy-coded variable (either one we created before or a new one).

To construct your model don't forget to load in the statsmodels api:

```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
```

_I'm going to engineer a new dummy variable for 'HUGE houses'.  Those whose square footage is 3 (positive) standard deviations away from the mean._
```
Mean = 1315
STD = 853
Huge Houses > 3775 sq ft
```

In [ ]:
shd['Huge_homes'] = (shd['sq__ft'] > 3775).astype(int)

In [ ]:
shd['Huge_homes'].value_counts()

In [ ]:
shd.head()

In [ ]:
from sklearn.linear_model import LinearRegression

X = shd[['sq__ft','beds','baths','Huge_homes']].values
y = shd['price'].values

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

### 12. Plot the true price vs the predicted price to evaluate your MLR visually.

> **Tip:** with seaborn's `sns.lmplot` you can set `x`, `y`, and even a `hue` (which will plot regression lines by category in different colors) to easily plot a regression line.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

shd['y_pred'] = y_pred
sns.lmplot(x='price', y='y_pred', data=shd, hue='Huge_homes',)

### 13. List the five assumptions for an MLR model. 

Indicate which ones are the same as the assumptions for an SLR model. 

**SLR AND MLR**:  

- *Linearity: Y must have an approximately linear relationship with each independent X_i.*
- *Independence: Errors (residuals) e_i and e_j must be independent of one another for any i != j.*
- *Normality: The errors (residuals) follow a Normal distribution.*
- *Equality of Variances: The errors (residuals) should have a roughly consistent pattern, regardless of the value of the X_i. (There should be no discernable relationship between X_1 and the residuals.)*

**MLR ONLY**:  
- *Independence Part 2: The independent variables X_i and X_j must be independent of one another for any i != j*





### 14. Pick at least two assumptions and articulate whether or not you believe them to be met  for your model and why.

In [ ]:
# Normality:  Do the Residual Errors follow a normal distribution?

# I believe all those properties with 0 values are causing the Y intercept to be higher up
# resulting in a less steep slope, thus creating areas where residual error is higher.

shd['Residuals'] = shd['price'] - shd['y_pred']
sns.distplot(shd['Residuals'])

In [ ]:
# The errors are more or less skewed to the right, but do show approximate normality otherwise.

In [ ]:
# Equality of variance.  

# I believe all the observations with 0 sq ft are obscuring the predictive 
# trend so as prices increase error is also going to as well.

sns.lmplot(x='sq__ft', y='Residuals', data=shd)

### 15. [Bonus] Generate a table showing the point estimates, standard errors, t-scores, p-values, and 95% confidence intervals for the model you built. 

**Write a few sentences interpreting some of the output.**

> **Hint:** scikit-learn does not have this functionality built in, but statsmodels does in the `summary` method.  To fit the statsmodels model use something like the following.  There is one big caveat here, however!  `statsmodels.OLS` does _not_ add an intercept to your model, so you will need to do this explicitly by adding a column filled with the number 1 to your X matrix

```python
import statsmodels.api as sm

# The Default here is Linear Regression (ordinary least squares regression OLS)
model = sm.OLS(y,X).fit()
```

In [52]:
# Importing the stats model API
import statsmodels.api as sm


# Setting my X and y for modeling
shd['intercept'] = 1
X = shd[['intercept','sq__ft','beds','baths','Huge_homes']]
y = shd['price']

# The Default here is Linear Regression (ordinary least squares regression OLS)
model = sm.OLS(y,X).fit()

In [19]:
X.isnull()

NameError: name 'X' is not defined

In [53]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.194
Model:                            OLS   Adj. R-squared:                  0.191
Method:                 Least Squares   F-statistic:                     58.99
Date:                Fri, 04 May 2018   Prob (F-statistic):           1.19e-44
Time:                        10:00:42   Log-Likelihood:                -12938.
No. Observations:                 984   AIC:                         2.589e+04
Df Residuals:                     979   BIC:                         2.591e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   1.251e+05   9766.956     12.811      0.000    1.06e+05    1.44e+05
sq__ft         9.4257      6.991      1.348      0.178      -4.293      23.144
beds       -3898.9895   5966.047     -0.654      0.514   -1.56e+04    7808.722
baths       5.977e+04   8405.687      7.110      0.000    4.33e+04    7.63e+04
Huge_homes  1.748e+05   4.29e+04      4.074      0.000    9.06e+04    2.59e+05
==============================================================================
Omnibus:                      231.522   Durbin-Watson:                   0.432
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              548.052
Skew:                           1.258   Prob(JB):                    9.82e-120
Kurtosis:                       5.654   Cond. No.                     1.69e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.69e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

---

> The material following this point can be completed after the first lesson on Tuesday.

---

### 16. Regression Metrics

Implement a function called `r2_adj()` that will calculate $R^2_{adj}$ for a model. 

In [58]:
def r2_adj(y_true, y_preds, p):
    n = len(y_true)
    y_mean = np.mean(y_true)
    numerator = np.sum(np.square(y_true - y_preds)) / (n - p - 1)
    denominator = np.sum(np.square(y_true - y_mean)) / (n - 1)
    return 1 - numerator / denominator

### 17. Metrics, metrics, everywhere...

Write a function to calculate and print or return six regression metrics.  Use other functions liberally, including those found in `sklearn.metrics`.

In [62]:
import sklearn.metrics as metrics
import numpy as np

def lord_of_the_metrics(y_true, y_pred, p):
    mse = metrics.mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    msle = metrics.mean_squared_log_error(y_true, y_pred)
    mae = metrics.median_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    r2a = r2_adj(y_true, y_pred, p)
    
    print('Mean squared error      = ', mse)
    print('Root mean squared error = ', rmse)
    print('Mean squared log error  = ', msle)
    print('Median absolute error   = ', mae)
    print('R^2                     = ', r2)
    print('Adjusted R^2            = ', r2a)

### 18. Model Iteration

Evaluate your current home price prediction model by calculating all six regression metrics.  Now adjust your model (e.g. add or take away features) and see how to metrics change.

In [63]:
lord_of_the_metrics(shd['price'], y_pred, X.shape[1])

Mean squared error      =  15426429002.706516
Root mean squared error =  124203.17629878278
Mean squared log error  =  0.8485426890446842
Median absolute error   =  77320.60603709458
R^2                     =  0.19421335759663194
Adjusted R^2            =  0.19009379398516268


In [69]:
shd.columns

Index(['street', 'city', 'zip', 'state', 'beds', 'baths', 'sq__ft', 'type',
       'sale_date', 'price', 'latitude', 'longitude', 'Pred', 'Residuals',
       'Condo', 'Multi-Family', 'Residential', 'Huge_homes', 'y_pred',
       'intercept'],
      dtype='object')

In [75]:
features = ['beds', 'baths', 'sq__ft','Condo', 'Multi-Family', 'Residential']
X = shd[features].values
y = shd['price'].values

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

In [76]:
lord_of_the_metrics(shd['price'], y_pred, X.shape[1])

Mean squared error      =  15388880505.856361
Root mean squared error =  124051.92665112606
Mean squared log error  =  0.8504445997137092
Median absolute error   =  77337.55541266792
R^2                     =  0.19617467198759508
Adjusted R^2            =  0.19123818072037424


### 19. Bias vs. Variance

At this point, do you think your model is high bias, high variance or in the sweet spot?  If you are doing this after Wednesday, can you provide evidence to support your belief?

In [80]:
from sklearn.model_selection import cross_val_score


cv_scores = cross_val_score(model, X, y)
print(cv_scores)
print(np.mean(cv_scores))

[ 0.0840832   0.10310967 -0.14280638]
0.014795496895109617
